In [2]:
import os
import xml.etree.ElementTree as ET

In [11]:
input_folder = r"C:\Users\ANSHUL M\Downloads\RDD2022_India\India\val\annotations\xmls"
output_folder = r"C:\Users\ANSHUL M\Downloads\RDD2022_India\India\val\annotations\labels"
os.makedirs(output_folder, exist_ok=True) 

In [12]:
class_name_to_id = {
    "D00": 0,
    "D10": 1,
    "D20": 2,
    "D40": 3,
    # Add more classes if needed...
}

In [13]:
for file_name in os.listdir(input_folder):
    if not file_name.endswith(".xml"):
        continue  # Skip non-XML files

    xml_path = os.path.join(input_folder, file_name)
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Retrieve image size to normalize bounding boxes
    size = root.find("size")
    if size is None:
        print(f"[Warning] No <size> tag in {xml_path}, skipping.")
        continue

    width = float(size.find("width").text)
    height = float(size.find("height").text)

    # The YOLO text file we will write
    txt_file_name = file_name.replace(".xml", ".txt")
    txt_path = os.path.join(output_folder, txt_file_name)

    # A list to hold each bounding box line
    yolo_lines = []

    # 4) Parse each <object> tag in the XML
    for obj in root.findall("object"):
        class_name = obj.find("name").text.strip()
        if class_name not in class_name_to_id:
            print(f"[Warning] Class '{class_name}' not in class_name_to_id. Skipping.")
            continue

        class_id = class_name_to_id[class_name]

        bndbox = obj.find("bndbox")
        xmin = float(bndbox.find("xmin").text)
        ymin = float(bndbox.find("ymin").text)
        xmax = float(bndbox.find("xmax").text)
        ymax = float(bndbox.find("ymax").text)

        # Convert to YOLO (x_center, y_center, width, height)
        x_center = (xmin + xmax) / 2.0
        y_center = (ymin + ymax) / 2.0
        box_width = (xmax - xmin)
        box_height = (ymax - ymin)

        # Normalize by image width/height
        x_center /= width
        y_center /= height
        box_width /= width
        box_height /= height

        # YOLO format: class_id x_center y_center width height
        yolo_line = f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}"
        yolo_lines.append(yolo_line)

    # 5) Write out the .txt file
    with open(txt_path, "w") as f:
        for line in yolo_lines:
            f.write(line + "\n")

    print(f"[INFO] Wrote {txt_path}")

[INFO] Wrote C:\Users\ANSHUL M\Downloads\RDD2022_India\India\val\annotations\labels\India_000081.txt
[INFO] Wrote C:\Users\ANSHUL M\Downloads\RDD2022_India\India\val\annotations\labels\India_000294.txt
[INFO] Wrote C:\Users\ANSHUL M\Downloads\RDD2022_India\India\val\annotations\labels\India_000326.txt
[INFO] Wrote C:\Users\ANSHUL M\Downloads\RDD2022_India\India\val\annotations\labels\India_000347.txt
[INFO] Wrote C:\Users\ANSHUL M\Downloads\RDD2022_India\India\val\annotations\labels\India_000349.txt
[INFO] Wrote C:\Users\ANSHUL M\Downloads\RDD2022_India\India\val\annotations\labels\India_000756.txt
[INFO] Wrote C:\Users\ANSHUL M\Downloads\RDD2022_India\India\val\annotations\labels\India_000797.txt
[INFO] Wrote C:\Users\ANSHUL M\Downloads\RDD2022_India\India\val\annotations\labels\India_000840.txt
[INFO] Wrote C:\Users\ANSHUL M\Downloads\RDD2022_India\India\val\annotations\labels\India_000909.txt
[Warning] Class 'D44' not in class_name_to_id. Skipping.
[INFO] Wrote C:\Users\ANSHUL M\Dow